# TASK 1: WEB SCRAPING – The data has been tabulated, cleaned, and prepared for Exploratory Data Analysis (EDA).

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import sqlite3
import re

In [4]:
def get_html(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Sayta daxil olunmadı: Status kodu {response.status_code}")

In [6]:
def parse_table(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", class_="wikitable")
    if table is None:
        raise ValueError("Cədvəl tapılmadı. HTML quruluşu dəyişmiş ola bilər.")

    rows = table.find_all("tr")
    data = []

    for row in rows[1:]:
        cols = row.find_all(["th", "td"])
        if len(cols) >= 5:
            try:
                rank = cols[0].text.strip()
                film = cols[1].text.strip()
                year = cols[2].text.strip()
                gross = cols[3].text.strip()
                distributor = cols[4].text.strip()
                data.append([rank, film, year, gross, distributor])
            except IndexError:
                continue
    return data


In [8]:
def get_html(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Sayta daxil olunmadı: Status kodu {response.status_code}")

In [10]:
def parse_table(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", class_="wikitable")
    if table is None:
        raise ValueError("Cədvəl tapılmadı. HTML quruluşu dəyişmiş ola bilər.")

    rows = table.find_all("tr")
    data = []

    for row in rows[1:]:
        cols = row.find_all(["th", "td"])
        if len(cols) >= 5:
            try:
                rank = cols[0].text.strip()
                film = cols[1].text.strip()
                year = cols[2].text.strip()
                gross = cols[3].text.strip()
                distributor = cols[4].text.strip()
                data.append([rank, film, year, gross, distributor])
            except IndexError:
                continue
    return data

In [12]:
def save_to_csv(data, filename):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Rank", "Film", "Year", "Gross", "Distributor"])
        writer.writerows(data)

In [14]:
def clean_dataframe(df):
    df.dropna(subset=["Film", "Gross"], inplace=True)

    # Clean the Gross column by removing symbols and converting it to a numeric type
    df["Gross"] = (
        df["Gross"]
        .astype(str)
        .str.replace(r"[^0-9.]", "", regex=True)
        .replace("", None)
        .astype(float)
    )

    # Clean the movie titles
    df["Film"] = (
        df["Film"]
        .astype(str)
        .str.replace(r"[\u2020\$SMT]+", "", regex=True)
        .str.strip()
    )

    # Extract the year from the date and convert it to an integer
    df["Year"] = (
        df["Year"]
        .astype(str)
        .str.extract(r"(\d{4})")
        .astype(float)
        .astype("Int64")
    )

    # Clean the Distributor names
    df["Distributor"] = df["Distributor"].astype(str).str.strip()

    return df


In [16]:
def save_to_sqlite(df, db_name):
    conn = sqlite3.connect(db_name)
    df.to_sql("films", conn, if_exists="replace", index=False)
    conn.close()

In [18]:
def main():
    url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
    html = get_html(url)
    data = parse_table(html)
    csv_file = "highest_grossing_films.csv"
    save_to_csv(data, csv_file)

    df = pd.read_csv(csv_file)
    df = clean_dataframe(df)
    save_to_sqlite(df, "films.db")
    print(df.head())

In [20]:
df = pd.read_csv("highest_grossing_films.csv")
df.head()

,Rank,Film,Gross,Distributor
0,1,Avatar,2.923706e+09,2009
1,2,Avengers: Endgame,2.797501e+09,2019
2,3,Avatar: he Way of Water,2.320250e+09,2022
3,4,itanic,2.257845e+09,1997
4,5,Ne Zha 2,2.198880e+09,2025


In [22]:
# If the Year column does not exist, add it
if "Year" not in df.columns:
    df["Year"] = None

In [24]:
#Swap the positions of the Distributor and Year columns (using remove() to rearrange the list of columns)
df = df.rename(columns={"Distributor": "Year", "Year": "Distributor"})

In [26]:
# First, convert to string type
df["Distributor"] = df["Distributor"].astype(str)

In [28]:
# If the value in the Distributor column is just 4 digits (i.e., given as a year), set it to None
df["Distributor"] = df["Distributor"].where(~df["Distributor"].str.contains(r'^\d{4}$', na=False), None)

In [30]:
df["Year"] = df["Year"].astype("Int64")  # Use Pandas' nullable integer type (Int64) introduced in version 1.0+
df["Distributor"] = df["Distributor"].astype(str).str.strip()

In [32]:
df.columns

Index(['Rank', 'Film', 'Gross', 'Year', 'Distributor'], dtype='object')

In [34]:
df.head()

,Rank,Film,Gross,Year,Distributor
0,1,Avatar,2.923706e+09,2009,None
1,2,Avengers: Endgame,2.797501e+09,2019,None
2,3,Avatar: he Way of Water,2.320250e+09,2022,None
3,4,itanic,2.257845e+09,1997,None
4,5,Ne Zha 2,2.198880e+09,2025,None


In [36]:
df.dropna(subset=["Film", "Gross"], inplace=True)

In [38]:
df.isnull().sum()

Rank           0
Film           0
Gross          0
Year           0
Distributor    0
dtype: int64

In [40]:
df.dtypes

Rank             int64
Film            object
Gross          float64
Year             Int64
Distributor     object
dtype: object

In [42]:
df["Gross"] = df["Gross"].replace(r"[^\d.]", "", regex=True).astype(float)

In [44]:
# Display the values in a readable format (for print only)
pd.options.display.float_format = '{:,.0f}'.format

In [46]:
df.dtypes

Rank             int64
Film            object
Gross          float64
Year             Int64
Distributor     object
dtype: object

In [48]:
df.Film = df.Film.str.replace(r"[†\$SMT]+", "", regex=True).str.strip()

In [50]:
df.head()

,Rank,Film,Gross,Year,Distributor
0,1,Avatar,"2,923,706,026",2009,None
1,2,Avengers: Endgame,"2,797,501,328",2019,None
2,3,Avatar: he Way of Water,"2,320,250,281",2022,None
3,4,itanic,"2,257,844,554",1997,None
4,5,Ne Zha 2,"2,198,880,000",2025,None


In [53]:
df=df.drop(columns=['Distributor'])

In [57]:
conn = sqlite3.connect("films.db")
df.to_sql("films", conn, if_exists="replace", index=False)
conn.close()


In [ ]:
df.to_csv('highest_grossing_films.csv', index=False)
